In [3]:
!pip uninstall keras
!pip install keras==2.1.5 
#!pip install tensorflow-gpu==1.6.0
!pip install 'h5py==2.10.0' --force-reinstall

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Would remove:
    /tensorflow-1.15.2/python3.7/Keras-2.3.1.dist-info/*
    /tensorflow-1.15.2/python3.7/docs/*
    /tensorflow-1.15.2/python3.7/keras/*
Proceed (y/n)? y
  Successfully uninstalled Keras-2.3.1
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into 

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import keras
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt 
from PIL import Image

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
#IF THE VERSION ARE NOT CORRECT RESTART RUNTIME. 
#Try also restart run, reinstall
# correct versions below
print(tf.__version__) #1.15.2
print(keras.__version__)  #2.1.5
print(h5py.__version__) # 2.10.0

1.15.2
2.1.5
2.10.0


In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
#The repo below is a popular repo that is mentioned repeatable for those who want to translate darknet models to keras models. Primarly yolov3
!git clone https://github.com/qqwweee/keras-yolo3 



Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 2.29 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [5]:
%cd keras-yolo3


/content/keras-yolo3


In [ ]:
#IMPORTING all weights and PASCAL training set. The above repo included a data preprocessing script that is used "voc_annotation.py"
#The pretrained weights are retrieved form here and the dataset
#!wget https://pjreddie.com/media/files/yolov3.weights
#!wget https://pjreddie.com/media/files/yolov3-tiny.weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1S1V-V0JtbQE6Z0LQmEMNPJJyJYgjFSsY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1S1V-V0JtbQE6Z0LQmEMNPJJyJYgjFSsY" -O fine_tune_yolov3.h5 && rm -rf /tmp/cookies.txt

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar
!python voc_annotation.py 
#USING REPO convert.py to convert from darknet model to keras model and save it a uploadable .h5 file
# !python  convert.py yolov3.cfg yolov3.weights model_data/yolov3_weights.h5
# !python convert.py yolov3-tiny.cfg yolov3-tiny.weights model_data/yolov3-tiny_weights.h5

In [7]:
from train import *
from yolo import *


In [8]:
#Preprocessing data parameters
input_shape = (416,416)
annotation_path_train = '2007_test.txt'
annotation_path_test = '2007_test.txt'
log_dir = 'logs/000/'
classes_path = 'model_data/voc_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

val_split = 0.1
training_lines = None
with open(annotation_path_train) as f:
  training_lines = f.readlines()

np.random.seed(10101)
np.random.shuffle(training_lines)
np.random.seed(None)
num_val = int(len(training_lines)*val_split)
num_train = len(training_lines) - num_val

batch_size = 16

#Pre-trained weights
path_teacher = "/content/keras-yolo3/model_data/yolov3_weights.h5"
path_student = "/content/keras-yolo3/model_data/yolov3-tiny_weights.h5"

#Fine-tuned_weights
path_teacher_fine_tuned = "/content/keras-yolo3/fine_tune_yolov3.h5"

#TODO FIX A STUDENT MODEL
# anchors_tiny_path = 'model_data/tiny_yolo_anchors.txt'
# anchors_tiny = get_anchors(anchors_tiny_path)

In [ ]:
def create_pre_trained_model(input_shape, anchors, num_classes, weights_path):
  #TODO INTRODUCE CONVERTERT
  #based on repo keras-yolo3 train.py  .
  #This allows to create models from the previously created .h5 file and train them.

  model = create_model(input_shape, anchors, num_classes, weights_path)
  return model


In [ ]:
def create_pre_trained_tiny_model(input_shape, anchors, num_classes, weight_path):
  #TODO INTRODUCE CONVERTER
  #based on repo keras-yolo3 train.py  .
  #This allows to create models from the previously created .h5 file and train them.

  model = create_tiny_model(input_shape, anchors, num_classes, weights_path)
  return model

In [9]:
def create_fine_tuned_model(weight_path, anchors_path, classes_path):
# Must change in generate method:
# load_model add arg: by_name = True

  defaults = {
        "model_path": weight_path,
        "anchors_path": anchors_path,
        "classes_path": classes_path,
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }
  model = YOLO(**defaults)
  return model


In [ ]:
model = create_fine_tuned_model(path_teacher_fine_tuned, anchors_path, classes_path)

In [ ]:
def train_model(model, training_lines, num_train, batch_size, input_shape, anchors, num_classes):
  #based on train.py in repo
  lines = training_lines

  logging = TensorBoard(log_dir=log_dir)
  checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
      monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
  early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
                                 

  for i in range(len(model.layers)):
    model.layers[i].trainable = True
  
  model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change


  print('Unfreeze all of the layers.')

  model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=150,  
    initial_epoch=100,
    callbacks=[logging, checkpoint, reduce_lr, early_stopping])

  return model

In [11]:
def test_generator(annotation_path, batch_size, input_shape, anchors, num_classes):
  #based on data_generator in train.py
  lines = None
 
  with open(annotation_path, 'r') as f:
    lines = f.readlines()
  n = len(lines)
  i = 0
  while True:
    image_data = []
    box_data = []
    for i in range(batch_size):
      image, box = get_random_data(lines[i], input_shape, random=False)
      image_data.append(image)
      box_data.append(box)
      i = (i+1) % n
    image_data = np.array(image_data)
    real_image = []
    for image in image_data:
      real_image.append(Image.fromarray(np.uint8(image*255)))
    box_data = np.array(box_data)
    y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)

    yield real_image, [*y_true]


In [12]:
generator = test_generator(annotation_path_test, batch_size, input_shape, anchors, num_classes)

In [13]:
images, y_trues = next(generator)

In [ ]:
# These classes and methods are based on https://keras.io/examples/vision/knowledge_distillation/

class Distiller(tf.keras.Model):
  def __init__(self, student, teacher):
    super(Distiller, self).__init__()
    self.teacher = teacher
    self.student = student

  def compile(sef, optimizer, metrices
              student_loss, distillation_loss, alpha=0.1, temperature=3):
    
     super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss = student_loss
        self.distillation_loss = distillation_loss
        self.alpha = alpha
        self.temperature = temperature
  
  def train(self, data):

    x,y = data
    teacher_predictions = self.teacher(x, training=False)

    with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)
           
            student_loss = self.student_loss(y, student_predictions)
            distillation_loss = self.distillation_loss(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

  def eval(self, data):

    x,y = data

    y_predictions = self.student(x, training=False)

    student_loss = self.student_loss(y, y_prediction)

    self.compiled_metrics.update_state(y, y_prediction)

    results = {m.name: me.result() for m in self.metrics}
    results.update({"student_loss": student_loss})
    return results


In [ ]:
teacher = create_fine_tuned_model()
student = create_tiny_pre_trained_model()
kd = Distiller(teacher, student)
kd.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred})
kd.train()

